<a href="https://colab.research.google.com/github/beaten-by-the-market/krxnewsscrap/blob/main/krxvsnxt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 파이썬 세팅하기

In [46]:
#-----------------------------------------------------------
#필요한 패키지 import
#-----------------------------------------------------------
import pandas as pd
import requests
from io import BytesIO

## 넥스트레이드 종목 시세 불러오기

In [47]:
url = 'https://www.nextrade.co.kr/brdinfoTime/brdinfoTimeList.do'

params = {
    "_search": False,
    "nd": 1741074199874,
    "pageUnit": 900,
    "pageIndex": 1,
    "sidx": "",
    "sord": "asc"
}


# POST 요청
response = requests.post(url, params=params)
data = response.json()  # JSON 데이터 변환

# 필요한 데이터 리스트 추출
brdinfo_list = data.get("brdinfoTimeList", [])

# 데이터프레임 전환
df1 = pd.DataFrame(brdinfo_list)
df_nxt = df1[['isuSrdCd', 'isuAbwdNm', 'mktNm', 'curPrc', 'contrastPrc', 'upDownRate', 'oppr', 'hgpr', 'lwpr', 'accTdQty', 'accTrval', 'tdhlYn']]

# 데이터프레임으로 변환할 컬럼 정의
df_nxt.columns = ['종목코드', '종목명', '시장구분', '현재가', '대비', '등락률', '시가', '고가', '저가', '거래량', '거래대금', '거래정지']

#종목코드의 첫 앞의 A를 제거하기
df_nxt = df_nxt.copy()  # 명시적으로 새로운 데이터프레임 생성
df_nxt['종목코드'] = df_nxt['종목코드'].str.replace('A', '', regex=True)

# 필요한 칼럼만 남기기
df_nxt = df_nxt[['종목코드', '종목명', '시장구분', '현재가', '등락률', '거래량', '거래대금']]

# 칼럼명 변경하기
df_nxt = df_nxt.rename(columns={'현재가': 'NXT현재가', '등락률': 'NXT등락률', '거래량': 'NXT거래량', '거래대금':'NXT거래대금'})

## 한국거래소 시세 불러오기

In [42]:
#-----------------------------------------------------------
#정보데이터시스템에서 불러오는 과정
#-----------------------------------------------------------

#[12001] 전종목 시세
#거래소 홈페이지에서 불러오기
gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp = {
    'locale' : 'ko_KR',
    'mktId': 'ALL',
    'trdDd': '20250304',
    'share': '1',
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT01501'
    }

headers = {'Referer' : 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

otp = requests.post(gen_otp_url, gen_otp, headers = headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_content = requests.post(down_url, {'code': otp}, headers = headers)


df_krx = pd.read_csv(BytesIO(down_content.content), encoding = 'EUC-KR')
df_krx['시장구분'] = df_krx['시장구분'].replace('KOSDAQ GLOBAL', 'KOSDAQ')
df_krx = df_krx.rename(columns=({'단축코드':'stock_code'}))

# 필요한 칼럼만 남기기
df_krx = df_krx[['종목코드', '종가', '등락률', '거래량', '거래대금', '시가총액']]

# 칼럼명 변경하기
df_krx = df_krx.rename(columns={'종가': 'KRX종가','등락률': 'KRX종가등락률', '거래량': 'KRX거래량', '거래대금': 'KRX거래대금'})

## 합치고 출력하기

In [52]:
# 합치기
df_agg = pd.merge(df_nxt, df_krx, on='종목코드', how='left')

# KRX 대비 NXT의 거래량 비중을 %로 계산하여 '비중'이라는 칼럼 만들기
df_agg['비중'] = 100*(df_agg['NXT거래량']/df_agg['KRX거래량'])

# 넥스트레이드 기준 날짜 및 시각 출력
if data.get("brdinfoTimeList"):
    now_date = data["brdinfoTimeList"][0].get("nowDd", "날짜 없음")
    now_time = data["brdinfoTimeList"][0].get("nowTime", "시간 없음")
    print(f"넥스트레이드 기준 날짜: {now_date}, 기준 시각: {now_time}")
else:
    print("데이터가 없습니다.")

# **추가: 전체 거래량 합산 및 비율 계산**
total_nxt_vol = df_agg['NXT거래량'].sum()
total_krx_vol = df_agg['KRX거래량'].sum()
nxt_vs_krx_ratio = (total_nxt_vol / total_krx_vol) * 100 if total_krx_vol > 0 else 0

# 상단에 정보 추가
print(f"📌 오늘 NXT는 KRX의 거래량 대비 **{nxt_vs_krx_ratio:.2f}%**를 기록했습니다.\n\n")
print(df_agg.to_markdown())
df_agg

넥스트레이드 기준 날짜: 20250304, 기준 시각: 1829
📌 오늘 NXT는 KRX의 거래량 대비 **16.29%**를 기록했습니다.


|    |   종목코드 | 종목명             | 시장구분   |   NXT현재가 |   NXT등락률 |   NXT거래량 |   NXT거래대금 |   KRX종가 |   KRX종가등락률 |   KRX거래량 |   KRX거래대금 |      시가총액 |     비중 |
|---:|-----------:|:-------------------|:-----------|------------:|------------:|------------:|--------------:|----------:|----------------:|------------:|--------------:|--------------:|---------:|
|  0 |     122870 | 와이지엔터테인먼트 | KOSDAQ     |       66400 |        5.4  |      207028 |   13662275800 |     66000 |            4.76 |      503997 |   32409235650 | 1233609234000 | 41.0772  |
|  1 |     120110 | 코오롱인더         | KOSPI      |       34400 |        2.99 |       36844 |    1260788175 |     34450 |            3.14 |      148940 |    5057190525 |  948032684950 | 24.7375  |
|  2 |     032640 | LG유플러스         | KOSPI      |       10580 |        0    |       67133 |     710048260 |     10500 |           -0.76 |      937660 |    9927955700 | 4584419290500 